In [1]:
import gymnasium as gym
from stable_baselines3 import SAC
import torch

env = gym.make('BipedalWalker-v3',
               hardcore=True,
               render_mode='human')

obs, _ = env.reset()

In [2]:
import sys
sys.path.append('/home/cy/MTSC/TodyNet/src')
from net import GNNStack
todeynet =  GNNStack(gnn_model_type='dyGIN2d', num_layers=3, 
            groups=4, pool_ratio=0.2, kern_size=[9,5,3], 
            in_dim=64, hidden_dim=128, out_dim=256, 
            seq_len=50, num_nodes=24, num_classes=2)
todeynet.load_state_dict(torch.load('/home/cy/MTSC/TodyNet/model/02.29_gpu0_dyGIN2d_BipedalWalkerHC_exp.pth'))

<All keys matched successfully>

In [3]:
model = SAC.load('/home/cy/newGYM/gymnasium_model/model/exp1130/BipedalWalkerHC-1e6-SAC-0.zip')
done = False
truncated = False
total_reward = 0
obs_record = []

while not done and not truncated:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    obs_record.append(obs)
    total_reward += reward
    env.render()
    if done:
        obs, _ = env.reset()
total_reward

78.82206947528505

In [4]:
import torch
obs_record = torch.as_tensor(obs_record)

/tmp/ipykernel_1433970/3567992264.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  obs_record = torch.as_tensor(obs_record)


In [5]:
obs_record.shape

torch.Size([664, 24])

In [6]:
temp = obs_record[-50:].reshape(-1,1,24,50)
temp.shape

torch.Size([1, 1, 24, 50])

In [7]:
pred = []
for i in range(0, len(obs_record)-50):
    temp = obs_record[i:i+50].reshape(-1,1,24,50)
    pred.append(todeynet(temp))

In [8]:
pred

[tensor([[-0.0715,  0.1297]], grad_fn=<AddmmBackward0>),
 tensor([[-0.1054,  0.1616]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0564,  0.1100]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0404,  0.0870]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.0480, -0.0094]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.1271, -0.0668]], grad_fn=<AddmmBackward0>),
 tensor([[-0.1157,  0.1764]], grad_fn=<AddmmBackward0>),
 tensor([[0.0065, 0.0455]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0307,  0.0693]], grad_fn=<AddmmBackward0>),
 tensor([[0.0149, 0.0702]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0258,  0.0881]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0267,  0.0683]], grad_fn=<AddmmBackward0>),
 tensor([[0.0031, 0.0503]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0245,  0.0748]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.1007, -0.0270]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.1572, -0.1302]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.1622, -0.1171]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0065,  0.0514]], grad_fn

In [13]:
X_train = torch.load('/home/cy/MTSC/TodyNet/data/UCR/BipedalWalkerHC/X_train.pt')
y_train = torch.load('/home/cy/MTSC/TodyNet/data/UCR/BipedalWalkerHC/y_train.pt')

In [19]:
output = todeynet(X_train[:200])

KeyboardInterrupt: 

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        print(maxk)
        batch_size = target.size(0)
        print(batch_size)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        print('correct', correct)

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [18]:
accuracy(output, target, topk=(1, ))

1
50
correct tensor([[ True, False,  True,  True,  True,  True, False,  True,  True,  True,
          True,  True,  True,  True,  True, False, False,  True, False,  True,
          True,  True, False,  True,  True,  True, False,  True,  True,  True,
          True, False,  True,  True,  True,  True,  True,  True, False,  True,
         False,  True,  True, False,  True,  True, False,  True,  True,  True]])


[tensor([76.])]